In [3]:
# # Load Pretrained model

# import keras
# from keras.models import Model
# from keras.applications import inception_v3
# from keras import backend as K
# from keras.preprocessing import image
# import tensorflow as tf

# K.set_learning_phase(0)

# input_shape = (1, 776, 776, 3)
# input_tensor = tf.placeholder(tf.float32, input_shape)

# model = inception_v3.InceptionV3(
#     include_top=False, weights='imagenet', input_tensor=input_tensor)

# # K.set_session(tf.session().run(model))

In [1]:
import keras
from keras.models import Model
from keras.applications import inception_v3
from keras import backend as K
from keras.preprocessing import image
import tensorflow as tf

K.set_learning_phase(0)


class DeepDream():
    
    def __init__(self, image_path):
        self.image_path = image_path
        self.sess = tf.Session()
        self.setting_sess()
        self.model = inception_v3.InceptionV3(
            include_top=False, weights='imagenet')
        self.model_weights = self.model.get_weights()

        self.layer_contributions = {'mixed2': 0.2,
                                    'mixed3': 3., 'mixed4': 2., 'mixed5': 1.5}
        self.loss = K.variable(0.)
        self.max_loss = 10.
        self.setting_sess()
        self.main()

    def setting_sess(self):
        K.set_session(self.sess)

    def loss_func(self):
        for name in self.layer_contributions:
            coef = self.layer_contributions[name]
            x = self.model.get_layer(name=name).output
            scaling = K.prod(K.cast(K.shape(x), 'float32'))
            self.loss += coef * K.sum(K.square(x[:, 2:-2, 2:-2, :])) / scaling

    def grad_loss_eval(self):
        self.dream = self.model.input
        
        self.loss_func()

        self.grads = K.gradients(self.loss, self.dream)[0]

        outputs = [self.loss, self.grads]
        self.fetch_loss_and_grads = K.Function(inputs=[self.dream], outputs=outputs)

    def eval_loss_and_grads(self, x):
        self.setting_sess()
        self.grad_loss_eval()
        outs = self.fetch_loss_and_grads([x])  # outputs
        loss_value = outs[0]
        grad_values = outs[1]
        return loss_value, grad_values

    def gradient_ascent(self, x, iterations, step, max_loss=None):

        for i in range(iterations):
            loss_value, grad_values = self.eval_loss_and_grads(x)

            if max_loss is not None and loss_value > max_loss:
                break
            print('...Loss value at', i, ':', loss_value)
            x += step * grad_values
        return x

    def resize_img(self, img, size):
        img = np.copy(img)
        factors = (1, float(size[0]) / img.shape[1],
                   float(size[1]) / img.shape[2], 1)
        return sp.ndimage.zoom(img, factors, order=1)

    def save_img(self, img, fname):
        pil_img = self.deprocess_image(np.copy(img))
        sp.misc.imsave(fname, pil_img)

    def preprocess_image(self, image_path):
        img = image.load_img(image_path)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = inception_v3.preprocess_input(img)

        return img

    def deprocess_image(self, x):

        x = x.reshape((x.shape[1], x.shape[2], 3))

        x /= 2.
        x += 0.5
        x *= 255.
        x = np.clip(x, 0, 255).astype("uint8")
        return x

    def main(self):
        
#         K.get_session().run(tf.global_variables_initializer())

#         uninit = K.get_session().run(tf.report_uninitialized_variables())
#         def assign_op():
#             for i, v in zip(uninit, self.model_weights):
#                 assign_op = i.assign(v)
        
#                 K.get_session().run(assign_op)
#         assign_op()
        
#         uninitialized_vars = []
#         for var in tf.global_variables():
#             try:
#                 K.get_session().run(var)
#             except tf.errors.FailedPreconditionError:
#                 uninitialized_vars.append(var)
                
#         for i in uninitialized_vars:
#             for v in self.model_weights:
#                 try :
#                     init_new_vars_op = tf.assign(i, v)
#                     K.get_session().run(init_new_vars_op)
#                 except:
#                     continue


        K.get_session().run(tf.keras.applications.InceptionV3(include_top=False, weights='imagenet'))

        step = 0.01
        num_octave = 3
        octave_scale = 1.4
        iterations = 30
        max_loss = 10.

        img = self.preprocess_image(self.image_path)

        original_shape = img.shape[1:3]
        successive_shapes = [original_shape]

        for i in range(1, num_octave):
            shape = tuple([int(dim / (octave_scale**i))
                           for dim in original_shape])
            successive_shapes.append(shape)

        successive_shapes = successive_shapes[::-1]  # 스케일 리스트

        original_img = np.copy(img)
        # starting from smallest image
        shrunk_original_img = self.resize_img(img, successive_shapes[0])

        for shape in successive_shapes:

            print('Processing image shape', shape)

            img = self.resize_img(img, shape)

            img = self.gradient_ascent(img, iterations=iterations,
                                  step=step, max_loss=max_loss)

            upscaled_shrunk_original_img = self.resize_img(
                shrunk_original_img, shape)

            same_size_original = self.resize_img(original_img, shape)

            lost_detail = same_size_original - upscaled_shrunk_original_img

            img += lost_detail

            shrunk_original_img = self.resize_img(original_img, shape)

        # plt.imshow(deprocess_image(img))
        # image.save_img('result_deepdream.png', deprocess_image(img))
        self.save_img(img, fname)

Using TensorFlow backend.


In [ ]:
deap_dream = DeepDream("./deepdream_parc.png")

In [3]:
model = tf.keras.applications.VGG16(include_top=False, weights='imagenet')

In [6]:
weights = model.get_weights()

In [14]:
np.array(weights).shape

(1, 26)

In [23]:
layers = model.layers

In [28]:
layers[0].get_weights()

[]

In [21]:
def preprocess_image(image_path):
        img = image.load_img(image_path)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = inception_v3.preprocess_input(img)
        return img

def forward_propagation(layers_output, W, b):
    """
    layers_output -> next_layers input 
    """
    return W(layers_output + b)

In [24]:
layers_input = preprocess_image("./deepdream_parc.png")

for layer in layers :
    W, b = layer.get_weights()
    layers_input = forward_propagation(layers_input, W, b)

ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
def forward_propagation(layers_output, W, b):
    """
    layers_output -> next_layers input 
    """
    
    return W(layers_output + b)
